In [1]:
from pandas import read_fwf
import numpy as np
import pandas as pd

In [2]:
# This script reads data files from the DHS survey
# https://dhsprogram.com/data/dataset/Zambia_Standard-DHS_2018.cfm
# and then outputs selected columns to a csv file

# Folder where data files are located
data_folder = '../Data/DHSSurvey/'
employee_survey_folder = 'ZMIR71FL/ZMIR71DT/'
# Choose the file prefix.
filename = 'ZMIR71FL'
filepath = data_folder + employee_survey_folder + filename + '.DTA'

In [3]:
# When labels are repeated, use the method below https://stackoverflow.com/questions/31782283/loading-stata-file-categorial-values-must-be-unique
with pd.io.stata.StataReader(filepath) as sr:
    value_labels = sr.value_labels()

df = pd.read_stata(
    filepath,
    convert_categoricals=False,
)

for col in value_labels:
    if col.lower() in df.columns:
        df[col.lower()].replace(value_labels[col], inplace=True)

In [4]:
# df

In [5]:
# label variable v005     "Women's individual sample weight (6 decimals)"
# label variable v024     "Region"
# label variable v025     "Type of place of residence"
# label variable v704     "Husband/partner's occupation"
# label variable v704a    "Husband/partner worked in last 7 days/12 months"
# label variable v705     "Husband/partner's occupation (grouped)"
# label variable v714     "Respondent currently working"
# label variable v714a    "Respondent has a job, but currently absent"
# label variable v716     "Respondent's occupation"
# label variable v717     "Respondent's occupation (grouped)"
# label variable v719     "Respondent works for family, others, self"
# label variable v721     "NA - Respondent works at home or away"
labels_services = ['v005','v024', 'v025', 'v704', 'v704a', 'v705', 'v714', 'v714a', 'v716','v717', 'v719', 'v721']
df_services = df[labels_services]

In [6]:
# Read the names of the columns from the .DO file and convert to dictionary
do = read_fwf(data_folder + employee_survey_folder + filename +'.DO', skiprows=2)

In [7]:
col_dict = dict(zip(do['caseid'], do['"Case Identification"']))

In [8]:
df_services = df_services.rename(columns=col_dict)

In [9]:
df_services

,"""Women's individual sample weight (6 decimals)""","""Region""","""Type of place of residence""","""Husband/partner's occupation""","""Husband/partner worked in last 7 days/12 months""","""Husband/partner's occupation (grouped)""","""Respondent currently working""","""Respondent has a job, but currently absent""","""Respondent's occupation""","""Respondent's occupation (grouped)""","""Respondent works for family, others, self""","""NA - Respondent works at home or away"""
0,1892890,eastern,rural,"subsistence farmers, fishers, hunters and gat...",worked last 7 days,agricultural - self employed,yes,NaN,market-oriented skilled agricultural workers,agricultural - self employed,for family member,NaN
1,1892890,eastern,rural,"subsistence farmers, fishers, hunters and gat...",worked last 12 months,agricultural - self employed,yes,NaN,sales workers,sales,self-employed,NaN
2,1892890,eastern,rural,"subsistence farmers, fishers, hunters and gat...",worked last 12 months,agricultural - self employed,no,no,"subsistence farmers, fishers, hunters and gat...",agricultural - self employed,for family member,NaN
3,1892890,eastern,rural,"subsistence farmers, fishers, hunters and gat...",worked last 12 months,agricultural - self employed,no,no,"subsistence farmers, fishers, hunters and gat...",agricultural - self employed,self-employed,NaN
4,1892890,eastern,rural,"subsistence farmers, fishers, hunters and gat...",worked last 12 months,agricultural - self employed,no,no,"subsistence farmers, fishers, hunters and gat...",agricultural - self employed,for family member,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
13678,691259,muchinga,rural,"subsistence farmers, fishers, hunters and gat...",worked last 7 days,agricultural - self employed,yes,NaN,"subsistence farmers, fishers, hunters and gat...",agricultural - self employed,self-employed,NaN
13679,691259,muchinga,rural,cleaners and helpers,worked last 7 days,household and domestic,yes,NaN,"subsistence farmers, fishers, hunters and gat...",agricultural - self employed,self-employed,NaN
13680,691259,muchinga,rural,not working and didn't work in last 12 months,didn't work last 12 months,did not work,no,no,not working and didn't work in last 12 months,not working,NaN,NaN
13681,691259,muchinga,rural,"subsistence farmers, fishers, hunters and gat...",worked last 7 days,agricultural - self employed,yes,NaN,"subsistence farmers, fishers, hunters and gat...",agricultural - self employed,self-employed,NaN


In [10]:
# Calculate the occupation share within each region and per type of area - women
occupation_share_women = df.groupby(['v024', 'v025', 'v717'])['v005'].sum().unstack(fill_value=0)/1000000
occupation_share_women = occupation_share_women.div(occupation_share_women.sum(axis=1), axis=0) * 100  # Calculate share as percentage
occupation_share_women = occupation_share_women.applymap('{:.2f}'.format)
occupation_share_women

v717                agricultural - self employed clerical  \
v024          v025                                          
central       rural                        34.54     0.06   
              urban                         0.72     1.06   
copperbelt    rural                        24.03     0.00   
              urban                         2.20     0.65   
eastern       rural                        34.10     0.14   
              urban                         9.01     0.73   
luapula       rural                        43.38     0.00   
              urban                        13.76     0.66   
lusaka        rural                         7.22     0.00   
              urban                         1.12     1.60   
muchinga      rural                        28.66     0.22   
              urban                         2.58     0.36   
north western rural                        24.40     0.00   
              urban                         5.81     2.30   
northern      rural                        33.24     0.00   
              urban                        10.96     0.58   
southern      rural                        23.65     0.18   
              urban                         0.60     0.64   
western       rural                        38.28     0.00   
              urban                         3.20     0.57   

v717                household and domestic not working  \
v024          v025                                       
central       rural                   1.77       46.06   
              urban                   4.93       53.55   
copperbelt    rural                   2.18       49.64   
              urban                   6.10       55.97   
eastern       rural                   1.44       39.89   
              urban                   3.51       40.21   
luapula       rural                   0.84       34.40   
              urban                   4.97       49.12   
lusaka        rural                   7.13       60.24   
              urban                   8.47       46.20   
muchinga      rural                   0.68       60.77   
              urban                   3.73       51.84   
north western rural                   0.89       55.43   
              urban                   2.49       54.06   
northern      rural                   0.88       49.39   
              urban                   4.95       45.08   
southern      rural                   3.17       50.61   
              urban                  17.63       36.19   
western       rural                   0.81       40.31   
              urban                   3.23       49.55   

v717                professional/technical/managerial  sales services  \
v024          v025                                                      
central       rural                              2.55   8.39     0.56   
              urban                              6.88  24.19     4.39   
copperbelt    rural                              2.05  13.59     1.27   
              urban                              6.30  17.63     4.63   
eastern       rural                              1.73   8.65     0.42   
              urban                              9.77  21.65     5.57   
luapula       rural                              1.86  13.59     0.40   
              urban                              5.32  18.48     2.69   
lusaka        rural                              4.56  12.79     1.37   
              urban                              7.61  20.58     4.56   
muchinga      rural                              0.29   6.89     0.19   
              urban                              4.11  24.75     6.43   
north western rural                              1.90  13.21     0.50   
              urban                              8.27  15.28     1.89   
northern      rural                              1.31  11.20     0.30   
              urban                              5.23  22.54     3.91   
southern      rural                              1.58  14.90     1

In [11]:
# Calculate the occupation share within each region - women - to compare with table 3.7.1 from the DHS report
occupation_share_women_DHSreport = df.groupby(['v024', 'v717'])['v005'].sum().unstack(fill_value=0)/1000000
occupation_share_women_DHSreport = occupation_share_women_DHSreport.drop(['not working'], axis=1)
occupation_share_women_DHSreport = occupation_share_women_DHSreport.div(occupation_share_women_DHSreport.sum(axis=1), axis=0) * 100  # Calculate share as percentage
occupation_share_women_DHSreport = occupation_share_women_DHSreport.applymap('{:.2f}'.format)
occupation_share_women_DHSreport

v717,agricultural - self employed,clerical,household and domestic,professional/technical/managerial,sales,services,skilled manual,unskilled manual
v024,,,,,,,,
central,45.85,0.75,5.41,7.67,26.19,3.48,0.75,9.92
copperbelt,11.52,1.24,12.40,12.74,38.04,9.30,3.39,11.36
eastern,51.28,0.36,2.86,4.64,17.26,1.83,1.84,19.94
luapula,58.60,0.25,2.91,4.31,23.72,1.51,0.86,7.85
lusaka,3.63,2.68,15.96,13.88,37.67,7.99,2.91,15.28
muchinga,57.32,0.60,3.13,2.56,25.43,3.47,1.98,5.51
north western,44.17,1.25,2.86,7.72,30.55,1.88,1.87,9.70
northern,55.83,0.23,3.31,4.08,26.20,1.99,1.26,7.10
southern,29.75,0.61,14.61,4.09,36.45,4.88,0.98,8.64


In [12]:
# Calculate the share of each occupation within each region and per type of area = men
occupation_share_men = df.groupby(['v024', 'v025', 'v705'])['v005'].sum().unstack(fill_value=0)/1000000
occupation_share_men = occupation_share_men.div(occupation_share_men.sum(axis=1), axis=0) * 100  # Calculate share as percentage
occupation_share_men = occupation_share_men.applymap('{:.2f}'.format)
occupation_share_men

v705                agricultural - self employed clerical did not work  \
v024          v025                                                       
central       rural                        53.29     0.24         5.45   
              urban                         5.70     0.37         4.77   
copperbelt    rural                        30.18     0.71        11.36   
              urban                         5.93     0.17         6.61   
eastern       rural                        37.28     0.41        18.94   
              urban                         9.33     1.94         7.17   
luapula       rural                        69.56     0.16         2.88   
              urban                        19.66     1.20         1.13   
lusaka        rural                        19.26     0.09        11.62   
              urban                         1.85     0.85         4.38   
muchinga      rural                        58.21     0.00        13.38   
              urban                         6.24     0.65         3.57   
north western rural                        29.44     0.16        26.89   
              urban                         5.87     3.63        10.67   
northern      rural                        69.42     0.52         4.73   
              urban                        28.51     0.00         4.75   
southern      rural                        33.98     0.26        20.77   
              urban                         3.58     0.26         8.28   
western       rural                        52.24     0.00        17.91   
              urban                         4.10     0.45         8.30   

v705                don't know household and domestic  \
v024          v025                                      
central       rural       0.33                   3.98   
              urban       0.00                   8.41   
copperbelt    rural       0.00                   7.31   
              urban       0.43                   7.16   
eastern       rural       0.43                   2.80   
              urban       0.52                   5.09   
luapula       rural       0.16                   1.33   
              urban       0.29                   3.91   
lusaka        rural       0.50                   6.74   
              urban       0.00                   5.69   
muchinga      rural       0.43                   1.04   
              urban       0.00                   3.90   
north western rural       0.00                   1.74   
              urban       0.25                   2.00   
northern      rural       0.63                   1.38   
              urban       2.81                   4.37   
southern      rural       0.00                   1.07   
              urban       0.00                   5.72   
western       rural       0.81                   0.93   
              urban       1.24                   5.48   

v705                professional/technical/managerial  sales services  \
v024          v025                                                      
central       rural                              4.39   6.17     3.15   
              urban                             14.81  16.20    10.95   
copperbelt    rural                              8.08   4.71     4.84   
              urban                             14.54   9.06     7.56   
eastern       rural                              3.12   8.97     2.41   
              urban                             12.56  16.97     5.82   
luapula       rural                              5.34   5.55     1.01   
              urban                             14.42  13.51     8.97   
lusaka        rural                             10.09   5.09     9.05   
              urban                             17.09  16.26     8.52   
muchinga      rural                              4.00   5.30     2.37   
              urban                             16.96  20.02     9.29   
north western rural                              6.17   6.12     3.55   
              urban  

In [14]:
# Save the two DataFrame to a CSV file
occupation_share_men.to_csv(data_folder + 'employee_survey_men.csv')
occupation_share_women.to_csv(data_folder + 'employee_survey_women.csv')